Mount Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Install Tensorflow Model

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Install Object Detection API

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Test Model

In [ ]:
!python models/research/object_detection/builders/model_builder_tf2_test.py

Prepare tfrecord - train

In [ ]:
!python 'drive/MyDrive/Colab Notebooks/tf2_captcha_test/my_create_pascal_tf_record.py' \
    --label_map_path='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/config/my_pascal_label_map.pbtxt' \
    --data_dir='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit' \
    --year=VOC2012 \
    --set=train \
    --output_path='drive/MyDrive/Colab Notebooks/tf2_captcha_test/pascal_train.record'

In [ ]:
!python 'drive/MyDrive/Colab Notebooks/tf2_captcha_test/my_create_pascal_tf_record.py' \
    --label_map_path='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/config/my_pascal_label_map.pbtxt' \
    --data_dir='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit' \
    --year=VOC2012 \
    --set=val \
    --output_path='drive/MyDrive/Colab Notebooks/tf2_captcha_test/pascal_val.record'

Start Tensorboard for monitoring

In [ ]:
%load_ext tensorboard
%tensorboard --logdir 'drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/train/train'

Train Model

In [ ]:
!python models/research/object_detection/model_main_tf2.py \
--pipeline_config_path='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/config/ssd_mobilenet_v2_320x320_coco17_tpu-8-colab.config' \
--model_dir='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/train' \
--alsologtostderr

Export Model. You will get your model in ../exported_model/saved_model path.

In [ ]:
!python models/research/object_detection/exporter_main_v2.py \
--input_type=image_tensor \
--pipeline_config_path='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/config/ssd_mobilenet_v2_320x320_coco17_tpu-8-colab.config' \
--trained_checkpoint_dir='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/train' \
--output_directory='drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/exported_model'

Load model and setting.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
def download_images():
    base_dir = 'drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/test_img/'
    filenames = ['captcha16.png', 'captcha17.png','captcha18.png']
    #filenames = ['captcha241.png']
    image_paths = []
    for filename in filenames:
        image_path = base_dir + filename
        image_path = pathlib.Path(image_path)
        image_paths.append(str(image_path))
    return image_paths

IMAGE_PATHS = download_images()
print(IMAGE_PATHS)

PATH_TO_MODEL_DIR = 'drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/exported_model'
print(PATH_TO_MODEL_DIR)
PATH_TO_LABELS = 'drive/MyDrive/Colab Notebooks/tf2_captcha_test/VOCdevkit/config/my_pascal_label_map.pbtxt'
print(PATH_TO_LABELS)

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

Run your model.

In [ ]:
%matplotlib inline
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#import warnings
#warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    return np.array(Image.open(path))

for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')
    image_np = load_image_into_numpy_array(image_path)

    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=10,
          min_score_thresh=.50,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
plt.show()
print('Draw images done')